# Free LLM Models for Entity and Relationship Extraction

This notebook explores free/open-source LLM alternatives to OpenAI for entity and relationship extraction in news articles.

## Model Options Evaluated:
1. **Ollama** (Local inference)
   - Llama 2/3 variants
   - Mistral models
   - CodeLlama for structured output

2. **Hugging Face Transformers** (Local/Cloud)
   - BERT-based NER models
   - GPT-2/GPT-Neo variants
   - Specialized entity extraction models

3. **Local Inference Servers**
   - llama.cpp
   - text-generation-webui
   - vLLM for faster inference

In [ ]:
import json
import requests
import re
from typing import List, Dict, Optional
from datetime import datetime
import os

# Try to import optional dependencies
try:
    import ollama
    ollama_available = True
except ImportError:
    print("Ollama not available. Install with: pip install ollama")
    ollama_available = False

try:
    from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
    transformers_available = True
except ImportError:
    print("Transformers not available. Install with: pip install transformers torch")
    transformers_available = False

# Load sample news data
def load_sample_news():
    """Load sample Scranton news for testing."""
    try:
        with open('../data/daily/scranton_news_2025-06-25.json', 'r') as f:
            news_data = json.load(f)
        return news_data.get('articles', [])[:3]  # Test with first 3 articles
    except FileNotFoundError:
        # Fallback sample data
        return [
            {
                "title": "Mayor Cognetti Announces New Infrastructure Project in Scranton",
                "description": "Scranton Mayor Paige Cognetti announced a $2 million infrastructure improvement project targeting Providence Road. The project will improve drainage and road conditions for residents."
            }
        ]

sample_articles = load_sample_news()
print(f"Loaded {len(sample_articles)} sample articles for testing")

## 1. Ollama Local LLM Integration

Ollama provides easy local inference for various open-source models.

In [ ]:
class OllamaExtractor:
    """Entity and relationship extraction using Ollama."""
    
    def __init__(self, model="llama3.2:3b"):
        self.model = model
        self.available = ollama_available
        
        if self.available:
            try:
                # Test if model is available
                ollama.show(model)
                print(f"✓ Ollama model {model} is available")
            except Exception as e:
                print(f"⚠️ Ollama model {model} not found. Pull with: ollama pull {model}")
                self.available = False
    
    def extract_entities(self, text: str) -> List[Dict]:
        """Extract named entities using Ollama."""
        if not self.available:
            return []
        
        prompt = f"""Extract named entities from this news text. Return only a JSON array of entities with 'name' and 'type' fields. Types should be: PERSON, ORGANIZATION, LOCATION, DATE, EVENT.

Text: {text}

JSON Response:"""
        
        try:
            response = ollama.generate(
                model=self.model,
                prompt=prompt,
                options={
                    "temperature": 0.1,
                    "top_p": 0.9,
                    "num_predict": 200
                }
            )
            
            # Extract JSON from response
            response_text = response['response']
            json_match = re.search(r'\[.*\]', response_text, re.DOTALL)
            
            if json_match:
                entities_json = json_match.group(0)
                entities = json.loads(entities_json)
                return entities
            else:
                print(f"No JSON found in response: {response_text}")
                return []
                
        except Exception as e:
            print(f"Ollama extraction failed: {e}")
            return []
    
    def extract_relationships(self, text: str, entities: List[Dict]) -> List[Dict]:
        """Extract relationships between entities."""
        if not self.available or not entities:
            return []
        
        entity_names = [e['name'] for e in entities]
        prompt = f"""Given these entities: {entity_names}
        
Extract relationships from this text. Return JSON array with 'from', 'to', 'type' fields. Relationship types: LOCATED_IN, WORKS_FOR, ANNOUNCED, PARTICIPATES_IN, PART_OF.

Text: {text}

JSON Response:"""
        
        try:
            response = ollama.generate(
                model=self.model,
                prompt=prompt,
                options={"temperature": 0.1, "num_predict": 150}
            )
            
            response_text = response['response']
            json_match = re.search(r'\[.*\]', response_text, re.DOTALL)
            
            if json_match:
                relationships = json.loads(json_match.group(0))
                return relationships
            return []
            
        except Exception as e:
            print(f"Ollama relationship extraction failed: {e}")
            return []

# Test Ollama extraction
ollama_extractor = OllamaExtractor()

if ollama_extractor.available and sample_articles:
    test_text = f"{sample_articles[0]['title']} {sample_articles[0]['description']}"
    print(f"\nTesting Ollama extraction on: {test_text[:100]}...")
    
    entities = ollama_extractor.extract_entities(test_text)
    print(f"Entities found: {entities}")
    
    relationships = ollama_extractor.extract_relationships(test_text, entities)
    print(f"Relationships found: {relationships}")
else:
    print("Ollama not available for testing")

## 2. Hugging Face Transformers Integration

Using specialized NER models from Hugging Face.

In [ ]:
class HuggingFaceExtractor:
    """Entity extraction using Hugging Face transformers."""
    
    def __init__(self):
        self.available = transformers_available
        self.ner_pipeline = None
        
        if self.available:
            try:
                # Use a lightweight NER model
                self.ner_pipeline = pipeline(
                    "ner", 
                    model="dbmdz/bert-large-cased-finetuned-conll03-english",
                    aggregation_strategy="simple"
                )
                print("✓ Hugging Face NER pipeline loaded")
            except Exception as e:
                print(f"⚠️ Failed to load HF model: {e}")
                self.available = False
    
    def extract_entities(self, text: str) -> List[Dict]:
        """Extract entities using Hugging Face NER."""
        if not self.available or not self.ner_pipeline:
            return []
        
        try:
            # Run NER pipeline
            ner_results = self.ner_pipeline(text)
            
            # Convert to our format
            entities = []
            for result in ner_results:
                entity_type = self.map_entity_type(result['entity_group'])
                entities.append({
                    'name': result['word'],
                    'type': entity_type,
                    'confidence': result['score']
                })
            
            # Remove duplicates and low confidence entities
            filtered_entities = []
            seen_names = set()
            
            for entity in entities:
                if (entity['confidence'] > 0.7 and 
                    entity['name'] not in seen_names and
                    len(entity['name']) > 2):
                    filtered_entities.append(entity)
                    seen_names.add(entity['name'])
            
            return filtered_entities
            
        except Exception as e:
            print(f"HuggingFace extraction failed: {e}")
            return []
    
    def map_entity_type(self, hf_type: str) -> str:
        """Map HuggingFace entity types to our schema."""
        mapping = {
            'PER': 'PERSON',
            'ORG': 'ORGANIZATION', 
            'LOC': 'LOCATION',
            'MISC': 'OTHER'
        }
        return mapping.get(hf_type, 'OTHER')
    
    def extract_relationships(self, text: str, entities: List[Dict]) -> List[Dict]:
        """Simple rule-based relationship extraction."""
        relationships = []
        
        # Simple patterns for relationship detection
        patterns = [
            (r'(\w+)\s+announced', 'ANNOUNCED'),
            (r'(\w+)\s+in\s+(\w+)', 'LOCATED_IN'),
            (r'Mayor\s+(\w+)', 'TITLE_OF')
        ]
        
        for pattern, rel_type in patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                if len(match.groups()) >= 2:
                    subj = match.group(1)
                    obj = match.group(2)
                    
                    # Check if entities exist
                    if any(e['name'].lower() == subj.lower() for e in entities):
                        if any(e['name'].lower() == obj.lower() for e in entities):
                            relationships.append({
                                'from': subj,
                                'to': obj,
                                'type': rel_type
                            })
        
        return relationships

# Test Hugging Face extraction
hf_extractor = HuggingFaceExtractor()

if hf_extractor.available and sample_articles:
    test_text = f"{sample_articles[0]['title']} {sample_articles[0]['description']}"
    print(f"\nTesting HuggingFace extraction on: {test_text[:100]}...")
    
    entities = hf_extractor.extract_entities(test_text)
    print(f"Entities found: {entities}")
    
    relationships = hf_extractor.extract_relationships(test_text, entities)
    print(f"Relationships found: {relationships}")
else:
    print("HuggingFace not available for testing")

## 3. Free API Services Integration

Some free cloud-based LLM services with reasonable limits.

In [ ]:
class FreeAPIExtractor:
    """Using free API services for extraction."""
    
    def __init__(self):
        # Check for API keys in environment
        self.huggingface_token = os.getenv('HUGGINGFACE_TOKEN')
        self.together_api_key = os.getenv('TOGETHER_API_KEY')
        
    def huggingface_inference_api(self, text: str) -> List[Dict]:
        """Use HuggingFace Inference API (free tier)."""
        if not self.huggingface_token:
            print("HuggingFace token not found. Set HUGGINGFACE_TOKEN env var")
            return []
        
        # Use free NER endpoint
        api_url = "https://api-inference.huggingface.co/models/dbmdz/bert-large-cased-finetuned-conll03-english"
        headers = {"Authorization": f"Bearer {self.huggingface_token}"}
        
        try:
            response = requests.post(api_url, headers=headers, json={"inputs": text})
            
            if response.status_code == 200:
                results = response.json()
                entities = []
                
                for result in results:
                    if result['score'] > 0.7:
                        entities.append({
                            'name': result['word'].replace('##', ''),
                            'type': result['entity_group'],
                            'confidence': result['score']
                        })
                
                return entities
            else:
                print(f"HF API error: {response.status_code}")
                return []
                
        except Exception as e:
            print(f"HF API extraction failed: {e}")
            return []
    
    def together_ai_extraction(self, text: str) -> List[Dict]:
        """Use Together AI free tier for extraction."""
        if not self.together_api_key:
            print("Together AI key not found. Set TOGETHER_API_KEY env var")
            return []
        
        # Together AI offers free credits for open-source models
        endpoint = "https://api.together.xyz/inference"
        headers = {
            "Authorization": f"Bearer {self.together_api_key}",
            "Content-Type": "application/json"
        }
        
        prompt = f"""Extract named entities from this news text. Return JSON format only:

{text}

JSON:"""
        
        payload = {
            "model": "togethercomputer/llama-2-7b-chat",
            "prompt": prompt,
            "max_tokens": 200,
            "temperature": 0.1
        }
        
        try:
            response = requests.post(endpoint, headers=headers, json=payload)
            
            if response.status_code == 200:
                result = response.json()
                generated_text = result['output']['choices'][0]['text']
                
                # Extract JSON from response
                json_match = re.search(r'\[.*\]', generated_text, re.DOTALL)
                if json_match:
                    entities = json.loads(json_match.group(0))
                    return entities
                return []
            else:
                print(f"Together AI error: {response.status_code}")
                return []
                
        except Exception as e:
            print(f"Together AI extraction failed: {e}")
            return []

# Test free API services
api_extractor = FreeAPIExtractor()
print(f"Free API services available:")
print(f"  HuggingFace: {'✓' if api_extractor.huggingface_token else '✗'}")
print(f"  Together AI: {'✓' if api_extractor.together_api_key else '✗'}")

## 4. Comprehensive Fallback Chain Implementation

Create a robust extraction system with multiple fallback options.

In [ ]:
class FreeLLMExtractionChain:
    """Comprehensive entity/relationship extraction with multiple fallbacks."""
    
    def __init__(self):
        self.extractors = {
            'ollama': OllamaExtractor(),
            'huggingface': HuggingFaceExtractor(),
            'api_services': FreeAPIExtractor()
        }
        
        # Determine available extractors
        self.available_extractors = []
        for name, extractor in self.extractors.items():
            if hasattr(extractor, 'available') and extractor.available:
                self.available_extractors.append(name)
            elif name == 'api_services':
                if extractor.huggingface_token or extractor.together_api_key:
                    self.available_extractors.append(name)
        
        print(f"Available extractors: {self.available_extractors}")
    
    def extract_entities_with_fallback(self, text: str) -> List[Dict]:
        """Try multiple extraction methods until one succeeds."""
        
        # Try Ollama first (local, fast)
        if 'ollama' in self.available_extractors:
            try:
                entities = self.extractors['ollama'].extract_entities(text)
                if entities:
                    print("✓ Used Ollama for entity extraction")
                    return entities
            except Exception as e:
                print(f"Ollama failed: {e}")
        
        # Try HuggingFace transformers (local)
        if 'huggingface' in self.available_extractors:
            try:
                entities = self.extractors['huggingface'].extract_entities(text)
                if entities:
                    print("✓ Used HuggingFace for entity extraction")
                    return entities
            except Exception as e:
                print(f"HuggingFace failed: {e}")
        
        # Try free API services
        if 'api_services' in self.available_extractors:
            api_extractor = self.extractors['api_services']
            
            if api_extractor.huggingface_token:
                try:
                    entities = api_extractor.huggingface_inference_api(text)
                    if entities:
                        print("✓ Used HuggingFace API for entity extraction")
                        return entities
                except Exception as e:
                    print(f"HF API failed: {e}")
            
            if api_extractor.together_api_key:
                try:
                    entities = api_extractor.together_ai_extraction(text)
                    if entities:
                        print("✓ Used Together AI for entity extraction")
                        return entities
                except Exception as e:
                    print(f"Together AI failed: {e}")
        
        # Final fallback: rule-based extraction
        print("⚠️ Using rule-based fallback for entity extraction")
        return self.rule_based_entity_extraction(text)
    
    def rule_based_entity_extraction(self, text: str) -> List[Dict]:
        """Fallback rule-based entity extraction."""
        entities = []
        
        # Simple patterns for entity detection
        patterns = {
            'PERSON': [r'\b[A-Z][a-z]+\s+[A-Z][a-z]+\b'],
            'LOCATION': [r'\bScranton\b', r'\bPennsylvania\b', r'\b[A-Z][a-z]+\s+Road\b'],
            'ORGANIZATION': [r'\b[A-Z][A-Z]+\b', r'\bDepartment\b', r'\bOffice\b']
        }
        
        for entity_type, pattern_list in patterns.items():
            for pattern in pattern_list:
                matches = re.finditer(pattern, text)
                for match in matches:
                    entity_name = match.group(0)
                    if len(entity_name) > 2 and not any(e['name'] == entity_name for e in entities):
                        entities.append({
                            'name': entity_name,
                            'type': entity_type,
                            'method': 'rule_based'
                        })
        
        return entities[:6]  # Limit to top 6
    
    def extract_relationships_with_fallback(self, text: str, entities: List[Dict]) -> List[Dict]:
        """Extract relationships with fallback methods."""
        
        # Try Ollama first
        if 'ollama' in self.available_extractors:
            try:
                relationships = self.extractors['ollama'].extract_relationships(text, entities)
                if relationships:
                    print("✓ Used Ollama for relationship extraction")
                    return relationships
            except Exception as e:
                print(f"Ollama relationship extraction failed: {e}")
        
        # Fallback to rule-based
        print("⚠️ Using rule-based fallback for relationship extraction")
        return self.rule_based_relationship_extraction(text, entities)
    
    def rule_based_relationship_extraction(self, text: str, entities: List[Dict]) -> List[Dict]:
        """Rule-based relationship extraction."""
        relationships = []
        entity_names = {e['name'].lower(): e for e in entities}
        
        # Simple relationship patterns
        patterns = [
            (r'(\w+)\s+announced', 'ANNOUNCED'),
            (r'(\w+)\s+in\s+(\w+)', 'LOCATED_IN'),
            (r'Mayor\s+(\w+)', 'HAS_TITLE'),
            (r'(\w+)\s+project.*in\s+(\w+)', 'OCCURS_IN')
        ]
        
        for pattern, rel_type in patterns:
            matches = re.finditer(pattern, text, re.IGNORECASE)
            for match in matches:
                groups = match.groups()
                if len(groups) >= 2:
                    subj = groups[0].lower()
                    obj = groups[1].lower()
                    
                    if subj in entity_names and obj in entity_names:
                        relationships.append({
                            'from': entity_names[subj]['name'],
                            'to': entity_names[obj]['name'],
                            'type': rel_type
                        })
        
        return relationships

# Test the complete fallback chain
extraction_chain = FreeLLMExtractionChain()

if sample_articles:
    test_text = f"{sample_articles[0]['title']} {sample_articles[0]['description']}"
    print(f"\nTesting complete extraction chain on: {test_text[:100]}...\n")
    
    # Extract entities with fallback
    entities = extraction_chain.extract_entities_with_fallback(test_text)
    print(f"\nFinal entities: {entities}")
    
    # Extract relationships with fallback
    relationships = extraction_chain.extract_relationships_with_fallback(test_text, entities)
    print(f"Final relationships: {relationships}")
    
    # Performance summary
    print(f"\n📊 Extraction Summary:")
    print(f"  Entities found: {len(entities)}")
    print(f"  Relationships found: {len(relationships)}")
    print(f"  Available methods: {len(extraction_chain.available_extractors)}")

## 5. Performance and Cost Comparison

Evaluate different approaches for accuracy, speed, and cost.

In [ ]:
def benchmark_extractors(articles: List[Dict]) -> Dict:
    """Benchmark different extraction methods."""
    
    results = {
        'methods_tested': [],
        'performance': {},
        'recommendations': []
    }
    
    extraction_chain = FreeLLMExtractionChain()
    
    for i, article in enumerate(articles[:2]):  # Test on first 2 articles
        text = f"{article.get('title', '')} {article.get('description', '')}"
        
        print(f"\n--- Testing Article {i+1} ---")
        print(f"Text: {text[:100]}...")
        
        # Time the extraction
        import time
        start_time = time.time()
        
        entities = extraction_chain.extract_entities_with_fallback(text)
        relationships = extraction_chain.extract_relationships_with_fallback(text, entities)
        
        extraction_time = time.time() - start_time
        
        results['performance'][f'article_{i+1}'] = {
            'entities_count': len(entities),
            'relationships_count': len(relationships),
            'extraction_time': round(extraction_time, 2),
            'entities': entities,
            'relationships': relationships
        }
        
        print(f"Extraction completed in {extraction_time:.2f}s")
        print(f"Found {len(entities)} entities, {len(relationships)} relationships")
    
    # Add method availability
    results['methods_tested'] = extraction_chain.available_extractors
    
    # Generate recommendations
    recommendations = []
    
    if 'ollama' in extraction_chain.available_extractors:
        recommendations.append("✓ Ollama: Best for privacy, local inference, no API costs")
    else:
        recommendations.append("⚠️ Consider installing Ollama for local LLM inference")
    
    if 'huggingface' in extraction_chain.available_extractors:
        recommendations.append("✓ HuggingFace: Good accuracy for NER tasks, local processing")
    else:
        recommendations.append("⚠️ Consider installing transformers for better NER accuracy")
    
    if len(extraction_chain.available_extractors) == 0:
        recommendations.append("⚠️ No advanced extractors available - using rule-based fallback only")
    
    recommendations.extend([
        "💡 For production: Use Ollama + HuggingFace as primary, rule-based as fallback",
        "💡 For development: API services offer good quality without local setup",
        "💡 For high volume: Local models (Ollama) avoid API rate limits and costs"
    ])
    
    results['recommendations'] = recommendations
    
    return results

# Run benchmark
if sample_articles:
    benchmark_results = benchmark_extractors(sample_articles)
    
    print("\n" + "="*60)
    print("📊 BENCHMARK RESULTS")
    print("="*60)
    
    print(f"\nMethods Available: {benchmark_results['methods_tested']}")
    
    print("\nPerformance Summary:")
    for article_key, stats in benchmark_results['performance'].items():
        print(f"  {article_key}: {stats['entities_count']} entities, {stats['relationships_count']} relationships ({stats['extraction_time']}s)")
    
    print("\nRecommendations:")
    for rec in benchmark_results['recommendations']:
        print(f"  {rec}")
else:
    print("No sample articles available for benchmarking")

## 6. Integration with Existing Pipeline

Show how to integrate free LLM extraction into the existing Scrantenna pipeline.

In [ ]:
def create_production_extractor():
    """Create a production-ready extractor for integration."""
    
    extractor_code = '''
# Add this to shorts/generate_shorts.py or create as separate module

class ProductionFreeLLMExtractor:
    """Production-ready free LLM entity/relationship extractor."""
    
    def __init__(self):
        self.ollama_available = self._check_ollama()
        self.hf_available = self._check_huggingface()
        
    def _check_ollama(self) -> bool:
        try:
            import ollama
            ollama.show("llama3.2:3b")  # Check if model exists
            return True
        except:
            return False
    
    def _check_huggingface(self) -> bool:
        try:
            from transformers import pipeline
            return True
        except:
            return False
    
    def extract_for_article(self, article: Dict) -> Dict:
        """Extract entities and relationships for a news article."""
        text = f"{article.get('title', '')} {article.get('description', '')}"
        
        # Extract entities
        entities = self._extract_entities_with_fallback(text)
        
        # Extract relationships  
        relationships = self._extract_relationships_with_fallback(text, entities)
        
        # Generate SVG visualization
        svg_graph = self._generate_svg_graph(entities, relationships)
        
        return {
            "entities": entities,
            "relationships": relationships, 
            "svg": svg_graph,
            "method": self._get_active_method()
        }
    
    def _extract_entities_with_fallback(self, text: str) -> List[Dict]:
        # Try Ollama first
        if self.ollama_available:
            try:
                return self._ollama_extract_entities(text)
            except:
                pass
        
        # Try HuggingFace
        if self.hf_available:
            try:
                return self._hf_extract_entities(text)
            except:
                pass
        
        # Fallback to rules
        return self._rule_based_entities(text)
    
    def _get_active_method(self) -> str:
        if self.ollama_available:
            return "ollama_llama3.2"
        elif self.hf_available:
            return "huggingface_bert"
        else:
            return "rule_based"
            
# Usage in generate_shorts.py:
# extractor = ProductionFreeLLMExtractor()
# graph_data = extractor.extract_for_article(article)
'''
    
    print("🔧 PRODUCTION INTEGRATION CODE:")
    print(extractor_code)
    
    # Save to file for easy integration
    with open('../shorts/free_llm_extractor.py', 'w') as f:
        f.write(extractor_code.strip())
    
    print("\n✅ Saved production extractor to: ../shorts/free_llm_extractor.py")
    print("\nTo integrate:")
    print("1. Copy the code to your project")
    print("2. Install dependencies: pip install ollama transformers torch")
    print("3. Pull Ollama model: ollama pull llama3.2:3b")
    print("4. Replace existing extraction in generate_shorts.py")

create_production_extractor()

## Summary and Recommendations

### Free LLM Options for Entity/Relationship Extraction:

1. **Ollama (Recommended)**
   - ✅ Completely free and local
   - ✅ No API keys or rate limits
   - ✅ Privacy-preserving (no data sent externally)
   - ✅ Good accuracy with Llama 3.2
   - ⚠️ Requires local GPU/CPU resources

2. **HuggingFace Transformers**
   - ✅ Excellent NER accuracy
   - ✅ Local processing
   - ✅ Well-established models
   - ⚠️ Limited to NER (entities only)

3. **Free API Services**
   - ✅ No local setup required
   - ✅ Good for development/testing
   - ⚠️ Rate limits and quotas
   - ⚠️ Data privacy concerns

### Cost Comparison:
- **OpenAI GPT**: ~$0.01-0.03 per 1000 tokens
- **Ollama**: $0 (one-time hardware cost)
- **HuggingFace Transformers**: $0 (local processing)
- **Free APIs**: $0 with usage limits

### Recommended Implementation:
1. **Primary**: Ollama with Llama 3.2 (3B or 7B)
2. **Secondary**: HuggingFace BERT for NER
3. **Fallback**: Rule-based extraction

This provides a robust, cost-effective pipeline that maintains quality while eliminating API dependency and costs.